# Data Setup

This chapter mostly focuses on data setup in order to prepare the `connect` function in `beaccess.py`.
This chapter differs from the others in that all of the code is set up and should work out of the box.
You do not need to input any information (unless, if you are an advanced user, have specific changes you want to make).

The first section prepares the data by creating a dictionary of lists.
Following that, there is a section that explains how to set up your backend,
and it contains a sub-section for a few different backends.

Backend sections will store the data created in the first section
into reachable locations, and then the final code block will modify `beaccess.py`
with access to that data (by writing re-writing the module programmatically).

## 0.1.0 Raw Data

We will be using mock farm data to work through the following chapters.

Our goal is to create [tabular data](https://en.wikipedia.org/wiki/Table_(database))
that we can store in a reachable location.
We can then use Ibis to take this data and perform several actions on it,
like [filtering](https://en.wikipedia.org/wiki/Filter_(higher-order_function)),
[aggregating](https://en.wikipedia.org/wiki/Aggregate_data),
or combining (through [joins](https://en.wikipedia.org/wiki/Join_(SQL))).

You don't need to know what these things are yet, as they will be explained in the next few chapters.

### 0.1.1 Summary

We will be using mock farm data to walk through our exercises and apply a practical application of Ibis.

In the following chapters, we will imagine that we are looking through and analyzing a set of farms.

Each farm has a record of harvest information--what crops were harvested,
when those crops were harvested,
who harvested them,
and where they were harvested.

Farms are split up by [tracts](https://en.wikipedia.org/wiki/Land_lot) (which are large plots of land),
and tracts are split up by [fields](https://en.wikipedia.org/wiki/Field_(agriculture)) (large plots of land that
we grow crops on).

These will be explained further in detail as we go through the tutorial,
so if something seems unclear or overwhelming, do not worry and just move on.
The gist of this is we are creating a set of tables that help us imagine what farm records might look like.

## 0.1.2 Data Creation

To prepare our tables, we will create a dictionary of lists.

Each list will be the data for our table, and the dictionary key for that list will be the table name.

In [1]:
files = list()
data = dict()

# Start a dictionary to store arguments for beaccess.py generation
beaccess_kwargs = dict()

# Farmer data - these are people that work on our farms
data['farmers'] = [
    ['farmer_id', 'farmer_name']
    ,[1, 'Alice']
    ,[2, 'Bob']
    ,[3, 'Mallory']
    ,[4, 'Pat']
    ,[5, 'Carol']
    ,[6, 'Carlos']
    ,[7, 'Charlie']
    ,[8, 'Yves']
]

# Farm data - this is general information about the farm
# name, location on our grid, address
data['farms'] = [
    [
        'farm_id','farm_name'
        ,'farm_address'
        ,'farm_origin_x','farm_origin_y'
    ],[
        1, 'North Farm'
        ,"{'street_address': '100 North St', 'city': 'City00', 'state': 'State00', 'postal_code': '00000'}"
        ,1, 7
    ],[
        2, 'East Farm'
        ,"{'street_address': '100 East St', 'city': 'City00', 'state': 'State00', 'postal_code': '00000'}"
        ,4, 2
    ],[
        3, 'South Farm'
        ,"{'street_address': '200 South St', 'city': 'City01', 'state': 'State01', 'postal_code': '00001'}"
        ,-2, -6
    ],[
        4, 'West Farm'
        ,"{'street_address': '300 West St', 'city': 'City02', 'state': 'State02', 'postal_code': '00002'}"
        ,-5, 2
    ]
]

# Farms are partitioned by tracts
# Tracts are plots of land on a farm that are broken up into fields
data['tracts'] = [
    ['tract_id', 'tract_farm_id']
    ,[1, 1]
    ,[2, 1]
    ,[3, 2]
    ,[4, 2]
    ,[5, 3]
    ,[6, 4]
]

# Tracts are partitioned by fields
# Fields are plots of land on a farm tract that are designated for growing crops
data['fields'] = [
    ['field_id', 'field_tract_id', 'field_vertices']
    ,[1, 1, [[0, 2], [1, 3], [2, 0], [3, 1]]]
    ,[2, 1, [[2, 2], [3, 2], [3, 1]]]
    ,[3, 2, [[0, 1], [-1, 0], [-2, 0], [-2, 1]]]
    ,[4, 3, [[0, 1], [1, 1], [1, 2], [0, 2]]]
    ,[5, 3, [[1, 0], [2, 0], [2, 3], [1, 3]]]
    ,[6, 3, [[2, 0], [3, 0], [3, 2], [2, 2]]]
    ,[7, 4, [[-1, -1], [0, -1], [0, -2]]]
    ,[8, 4, [[1, 0], [1, -2], [0, -2], [0, -1]]]
    ,[9, 5, [[1, 0], [2, 0], [1, -1], [1, -2], [-1, -2], [-1, -1], [-2, 0], [-1, 0], [0, -1]]]
    ,[10, 6, [[-1, 2], [0, 2], [0, 0], [-1, 0]]]
    ,[11, 6, [[0, 2], [1, 2], [1, 1], [0, 1]]]
]

# Date information for harvest dates
data['dates'] = [
    ['date_id', 'date_year', 'date_month', 'date_day']
    ,[1, 2021, 8, 4]
    ,[2, 2021, 8, 26]
    ,[3, 2022, 8, 3]
    ,[4, 2022, 8, 25]
]

# Crop information for crops harvested/planted
data['crops'] = [
    ['crop_id', 'crop_name', 'crop_unit']
    ,[1, 'soybean', 'bu']
    ,[2, 'potato', 'kg']
]

# Farmer groups - which farmers from our farmers table have worked together
data['farmer_groups'] = [
    ['farmer_group_id', 'farmer_group_members']
    ,[1, [1, 3, 4]]
    ,[2, [2, 4, 5]]
    ,[3, [6, 8]]
    ,[4, [1, 6]]
]

# Harvest detail - which field harvested how many of what crop on what date and which groups performed the harvest?
data['harvest'] = [
    ['harvest_id', 'harvest_field_id', 'harvest_farmer_group_id', 'harvest_crop_id', 'harvest_date_id', 'harvest_value']
    ,[1, 1, 1, 1, 1, 65.80]
    ,[2, 2, 1, 2, 2, 5750.00]
    ,[3, 3, 1, 1, 1, 59.85]
    ,[4, 4, 2, 2, 2, 10100.00]
    ,[5, 5, 2, 1, 1, 90.30]
    ,[6, 6, 2, 2, 2, 21000.00]
    ,[7, 7, 2, 2, 2, 5150.00]
    ,[8, 8, 2, 1, 1, 53.55]
    ,[9, 9, 3, 1, 1, 147.00]
    ,[10, 10, 4, 1, 1, 70.70]
    ,[11, 11, 4, 2, 2, 9600.00]
    ,[12, 1, 1, 2, 4, 22800.00]
    ,[13, 2, 1, 1, 3, 19.25]
    ,[14, 3, 1, 2, 4, 13050.00]
    ,[15, 4, 2, 1, 3, 31.15]
    ,[16, 5, 2, 2, 4, 33000.00]
    ,[17, 6, 2, 1, 3, 64.40]
    ,[18, 7, 2, 1, 3, 16.45]
    ,[19, 8, 2, 2, 4, 15000.00]
    ,[20, 9, 3, 2, 4, 38400.00]
    ,[21, 10, 4, 2, 4, 19800.00]
    ,[22, 11, 4, 1, 3, 34.30]
]

## 0.1.3 Storing Data

To use the data you've created, you need to store it where your backend can reach it.
Here are various methods of getting your data where it needs to be in order to connect to it.

Expand the section below for the backend you are using and follow its directions to populate
the data you will use to complete the next lessons.

### "DuckDB" / "pandas"

<div class="alert alert-block alert-warning">
<b>⚠️:</b> Alters Current Directory

    This creates a new directory in the current directory and writes several parquet files into it.
    If the paths exist, then those paths will be overwritten.
</div>

The code below will create a directory, `farm_data/`, and write several parquet files containing mock farm data.
DuckDB can connect to these files directly after they are written.

Pandas requires a dictionary of pandas.DataFrames, so we will read these parquets and then create that dictionary
to connect to the pandas backend.

In [2]:
import os
import glob

import pandas as pd

FARM_DATA_DIR = 'farm_data'


def write_data(data, fname, dname=FARM_DATA_DIR, fmt='parquet'):
    if not os.path.exists(dname):
        os.mkdir(dname)
    path = '.'.join([os.path.join(dname, fname), fmt])

    try:
        os.remove(path)
    except:
        pass

    if fmt in ['csv', 'txt']:
        with open(path, 'w') as f:
            for row in data:
                line = '|'.join(str(e) for e in row)
                f.write(line)
                f.write('\n')
    elif fmt in ['parquet']:
        df = pd.DataFrame(data[1:], columns=data[0])
        df.to_parquet(path, index=False)
    return path


for key in data:
    files.append(write_data(data[key], key))

beaccess_kwargs['duckdb'] = f"""
def duckdb_conn():
    conn = ibis.connect('duckdb://:memory:')

    dpath = '{os.path.join(os.path.abspath('.'), FARM_DATA_DIR)}'
    for file in os.listdir(dpath):
        if '.parquet' in file:
            conn.register(os.path.join(dpath, file), file.split('/')[-1].replace('.parquet', ''))
    return conn
"""

beaccess_kwargs['pandas'] = f"""
def pandas_conn():
    dfs = dict()
    dpath = '{os.path.join(os.path.abspath('.'), FARM_DATA_DIR)}'
    
    for file in os.listdir(dpath):
        if '.parquet' in file:
            dfs[file.replace('.parquet', '')] = pd.read_parquet(os.path.join(dpath, file))

    conn = ibis.pandas.connect(dfs)
    return conn
"""

### "Postgres"

<div class="alert alert-block alert-warning">
<b>⚠️:</b> Alters Postgres Instance public Schema

    This writes tables into a postgres `public` schema. If the table exists, that table will be overwritten.
</div>

If you are using the Postgres backend, then you will be using the `public` schema to store your data.

You should have a [.pgpass file](https://www.postgresql.org/docs/9.3/libpq-pgpass.html) in your home directory (`~/`).
This will make connecting simple since you won't need to type your password every time you connect.

Connect to your database using a connection string:

In [3]:
import ibis

cstring = 'postgres://<username>@<host>:<port>/<database>'

conn = ibis.connect(cstring)

#### Writing Using the Above Connection
After you've chosen your connection method and it has create table permissions, you will be able to connect and write your data.
You will use `pandas.DataFrame.to_sql` to write this data to your postgres instance's `public` schema:

In [4]:
import pandas as pd

# Array type function
arr_f = ibis.backends.postgres.sa.types.ARRAY

# Decimal type
dec_type = ibis.backends.postgres.sa.types.DECIMAL()

# Integer type
int_type = ibis.backends.postgres.sa.types.INT()

# Text type
str_type = ibis.backends.postgres.sa.types.Text()

# Need to make sure our data types are mapped correctly.
# Define schemas explicitly.
schemas = {
    'farmers': {
        'farmer_id': int_type
        ,'farmer_name': str_type
    }
    ,'farms': {
        'farm_id': int_type
        ,'farm_name': str_type
        ,'farm_address': str_type
        ,'farm_origin_x': int_type
        ,'farm_origin_y': int_type
    }
    ,'tracts': {
        'tract_id': int_type
        ,'farm_id': int_type
    }
    ,'fields': {
        'field_id': int_type
        ,'field_tract_id': int_type
        ,'field_vertices': arr_f(int_type)
    }
    ,'dates': {
        'date_id': int_type
        ,'date_year': int_type
        ,'date_month': int_type
        ,'date_day': int_type
    }
    ,'crops': {
        'crop_id': int_type
        ,'crop_name': str_type
        ,'crop_unit': str_type
    }
    ,'farmer_groups': {
        'farmer_group_id': int_type
        ,'farmer_group_members': str_type
    }
    ,'harvest': {
        'harvest_id': int_type
        ,'harvest_field_id': int_type
        ,'harvest_farmer_group_id': int_type
        ,'harvest_crop_id': int_type
        ,'harvest_date_id': int_type
        ,'harvest_value': dec_type
    }
}

for key in data:
    df = pd.DataFrame(data[key][1:], columns=data[key][0])
    df.to_sql(
        name=key
        ,con=conn.con.connect()
        ,if_exists='replace'
        ,index=False
        ,dtype=schemas[key]
    )

beaccess_kwargs['postgres'] = f"""
def postgres_conn():
    return ibis.connect('{cstring}')
"""

## 0.1.4 Write to `beaccess.py`

<div class="alert alert-block alert-warning">
<b>⚠️:</b> Alters Current Directory

    The next block creates a new .py file, beaccess.py, in the current directory and writes to it.
    If the file exists, then that file will be overwritten.
</div>

Each block above created a series of arguments that allow this section to create and modify `beaccess.py`.
By running the code below, you will create a function that allows you to establish an ibis connection
to the data you've created.

The next chapter will explain how to establish a connection, but every other chapter will use this connection.
Be sure to, at the very least, create a module `beaccess.py` with a function `connect` that returns an ibis
backend connection to your data.

In [5]:
default_backend = 'duckdb' if 'duckdb' in beaccess_kwargs.keys() else beaccess_kwargs.keys()[0]

fns = '\n'.join(beaccess_kwargs.values())

fdict = (
    'dict(\n        '
    + '\n        ,'.join(
        '='.join([k, v.split('\n')[1].replace('def ', '').replace(':', '')])
        for k, v in beaccess_kwargs.items()
    )
    + '\n    )'
)

script_string = f"""import os
import ibis
import pandas as pd

{fns}

def connect(backend='{default_backend}'):
    return {fdict}[backend]
"""

with open('./beaccess.py', 'w') as f:
    f.seek(0)
    f.truncate()

    f.write(script_string)

In [10]:
import beaccess as bea

for key in beaccess_kwargs:
    conn = bea.connect(key)
    print(key, conn.list_tables())

duckdb ['crops', 'dates', 'farmer_groups', 'farmers', 'farms', 'fields', 'harvest', 'tracts']
pandas ['tracts', 'harvest', 'farms', 'farmers', 'crops', 'fields', 'farmer_groups', 'dates']
postgres ['array_test', 'array_data', 'farmers', 'farms', 'tracts', 'fields', 'dates', 'crops', 'farmer_groups', 'harvest']
